# MICRO-452 Basics of mobile robotics

## Project Report

Team  13:

Wilhelm Laetitia,
Beuret Sylvain,
Labbe Victor,
Jaffal Oussama

### Table of Contents

1. [Introduction](#introduction)

2. [Global Set-up](#globalsetup)
    * [Map Set-up](#mapsetup)
    * [Libraries & constants](#libraries)
    * [Launch demo](#demo)
    * [Main.py structure](#main) 

3. [Computer vision](#computervision)
    * [Map detection](#mapdetection)
    * [ArUco detection](#arucodetection)
    * [Obstacles detection](#obstaclesdetection)
    * [Projection image](#projected-image)
    * [Results](#CVresults)
  

4. [Path planning](#pathplanning)
    * [Global Navigation](#globalnavigation)
    * [A* algorithm](#Aalgorithm)
    * [Implementation](#implementation)
    * [Results](#PPresults)

5. [Kalman Filtering](#filtering)
    * [General, filtering](#genfiltering)
    * [Mathematical model](#mathmodel)
    * [Filter](#filter)
    * [Results](#Kresults)

6. [Motion control](#motioncontrol)
    * [Orientation & angle error](#orientation)
    * [PD controler](#controler)
    * [Compute velocity](#velocity)
    * [Local navigation](#localnavigation)
    * [Results](#MCresults)

7. [Conclusion](#conclusion)

### Introduction <a class="anchor" id="introduction"></a>


This project is made in the frame of the EPFL course: Basics of Mobile Robotics (BoMR). The goal is to program a robot, the Thymio, to go from a specific point to another while avoiding the obstacles and surviving some kidnapping sessions. In order to achieve this, we used computer vision, filtering, global navigation and local navigation.

As you can see, we are on a car racing circuit, where our beloved thymio is racing against the clock.

Our program uses an overhead camera to detect the obstacles, the track, the thymio and the finishing line. The global navigation part, anaibles to do path planning to reach the line (goal). Finally we have a Kalman filter, using the robot's position from the camera and odometry, allowing the thymio to know and correct its position in real time, even with a hidden camera.

<div style="text-align: center;">
    <img src="image\schema-explication.jpeg" alt="Structure" title="Map" style="width:800px;"/>
</div>


<div style="text-align:center;">
     "Project structure"
</div>

<div style="text-align: center;">
<img src="image/thymio_futur.jpeg" alt="Setup Map" title="Map" style="width:200px;"/>
</div>


<div style="text-align:center;">
     Our thymio in the futur by AI
</div>

The videos are to big, so please refer to this one drive file to see our final results. https://1drv.ms/f/s!AvyzZel2mVd0qTD84Sf01JUx_LqD?e=luYhZV  

When the camera is on, a window is displayed.
To shut it down press the key 'q' when having the mouse on it BUT be careful as it will shut down the thread that might be feeding the main with data

The camera requires quite some time to launch. The terminal will provide you with the state of the process during all time !

-------

## Global set-up <a class="anchor" id="computervision"></a>

### Map Set-up <a class="anchor" id="setup"></a>

Our environment as be designed to ressemble a (basic) Mario Kart circuit. 
The road is in white and the sideways are in black. We have in addition (not displayed on picture) some 3D obstacles i.e. bad turtles staying on the road that we will need to avoid.
The 'L' obstacle can be turned to suit our mood. The demo works any ways.



<div style="text-align: center;">
<img src="image/setup_map.jpg" alt="Setup Map" title="Map" style="width:400px;"/> 
</div>


<div style="text-align:center;">
     Map seen from the camera
</div>


### Libraries <a class="anchor" id="libraries"></a>

Using OpenCv libraries, we get a livestream from the webcam in real time. We also use an initial frame of the video to draw the map that we need to plan the shortest path for the robot.

We also use librairies such as threading to achieve parallel computing of our codes, numpy for most mathematical operations and keyboard to manage our script.

Finally we use ClientASync from tdmclient to communication asynchronously with our Thymio. 

We have also implemented several classes to facilitate the integration of the different parts of our project: 
- Most objects our our simulation have a dedicated dataclass in the file utils/data.py (such as Map, Robot, Motors, etc). We used the library dataclasses in order to define those
- We also defined a Thymio class to interface communication with our robot in the file utils/communication.py

In [1]:
from timeit import default_timer as timer
from tdmclient import ClientAsync, aw
import time
import cv2
import cv2 as cv
import threading
import keyboard
import dataclasses

### Launch demonstration <a class="anchor" id="setup"></a>

To run our project, you need to download this [GitHub repository](https://github.com/Thebestvic07/BOMR_Project.git).

To launch our demo, you need to follow those steps:
1. Run the main.py file
2. First thing the script will do is connect to the Thymio (should print in the terminal)
3. Once connected to Thymio, it will draw and calibrate the map. *!! Don't put anything on the map except of the obstacles !!* 
4. Once this is done (should take max 1 min), the drawn map will appear in a window 
5. The script will then initialize the camera to provide real time feed in a second window (this can take a few minutes...)
6. When "searching thymio and goal" is written in the terminal, you can add both + some 3D turtles for the Thymio to avoid
7. Most objects present on the video will also be displayed on the map display alongside with some interesting features such as the optimal path to the goal 
8. Press "esc" in the terminal to end the demo and 'q' on the displayed windows to shut them down

### Main.py structure <a class="anchor" id="setup"></a>

We decided to use the different following thread in the main so several task can run in parallel.


##### 1. Camera Thread (<span style="text-decoration: underline;">*camera_thread*</span>):

**Function**: run_camera

**Purpose & Execution**: The run_camera function is executed in this thread. It continuously captures frames ($f=10Hz$), processes ArUco markers, and updates the robot and goal positions.

##### 2. Thymio Thread (<span style="text-decoration: underline;">*thymio_thread*</span>):
    
**Function**: update_thymio

**Purpose & Execution**: The update_thymio function is executed in this thread. It continuously reads motors variables ($f=10Hz$) from the Thymio, updating its state.

##### 3. Display Thread (<span style="text-decoration: underline;">*display_thread*</span>):

**Function**: display

**Purpose & Execution**: The display function is executed in this thread. It continuously updates the map displayed on a side window based on the environment's state.

##### 4. Main Thread:

**Function**: __main__

**Purpose**: The main thread orchestrates the overall control flow.

**Execution**: The script starts by initializing various objects, threads, and constants. It then enters the main loop, which handles tasks such as state estimation, path planning, motion updates, obstacle avoidance, and goal-reaching conditions. The main loop is controlled by a timer, and the script continuously monitors the state of the robot and the environment.

##### Note: The various Threads are so-called **Daemon threads**, so when the main program exits, the threads closes with it.



-------
## Vision <a class="anchor" id="computervision"></a>


After trying a lot of days to do shape detections, we decided to use Aruco markers (sub-library of OpenCV) to detect the thymio, and the goal. It proved to work a lot better than shape detection, especially in various light conditions. 

The obstacles are in black, so the thymio can go everywhere it is white.


#### Map detection<a class="anchor" id="mapdetection"></a>

The map detection is performed only once at the beginning of the process. We capture a single calibrating frame, with only the black ostacles and one ArUco. The ArUco marker serves the purpose of computing the length of one of its sides and store its value as the grid resolution, this way the map can be calibrated, before continuing the process, to ensure that the map's resolution remains independent of the height or angle of the camera. The frame is then used as a support for the display of the environment. This frame can be found in the Project repository : 'captured_frame.png'

The creation of the black and white image involves the is_black_cell function, which evaluates whether a grid cell in the captured image is predominantly black. This function takes an image as input and determines whether the cell is considered an obstacle or not. The process includes converting the image to grayscale, creating a binary mask using a specified threshold, calculating the percentage of black pixels, and comparing it to a threshold percentage (here, set to 10% to account for conditions of test environment). If the percentage of black pixels surpasses the threshold, the cell is categorized as black, signifying the presence of obstacles, otherwise, it is classified as white.


This process is done for each cell, and then we reconstruct the map using apply_grid method, by putting either black or white cells, and stack them horizontally and vertically, the purpose is to generate a simplified black and white representation of the environment, highlighting obstacle positions for further processing in Thymio navigation and obstacle avoidance. We also add various information in the according lists to have all the necessary data we need for the steps that follow.

In [ ]:
def is_black_cell(image):
    # Know if the grid cell contains more than 1/8 pixels who are black
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(image, 110, 255, cv2.THRESH_BINARY)

    # Calculate the percentage of black pixels in the image
    total_pixels = image.size
    black_pixels = total_pixels - cv2.countNonZero(binary_mask)
    percentage_black = (black_pixels / total_pixels) * 100

    # Set the threshold percentage
    threshold_percentage = 10  # Adjust this threshold as needed

    # Check if the percentage of black pixels exceeds the threshold
    if percentage_black > threshold_percentage:
        return True
    else:
        return False
    
# Build a map from an image
def apply_grid(image, grid_resolution):

    # Get image dimensions
    height, width = image.shape[:2]

    # Create black and white images
    create_black_image(grid_resolution)
    create_white_image(grid_resolution)

    # Get resolution of the grid
    x_cells = int(width / grid_resolution)
    y_cells = int(height / grid_resolution)

    # Creating a 2D list of grid cells
    map = [[0 for _ in range(y_cells)] for _ in range(x_cells)]
    obstacles = []
    internal_map = [[0 for _ in range(y_cells)] for _ in range(x_cells)]

    # Initializing variables
    obstacles = []
    new_image = list(range(y_cells))
    final_image = list(range(x_cells))

    for y in range(y_cells):
        for x in range(x_cells):
            # Define the bounding box for the current grid cell
            x_min, y_min = x*grid_resolution, y*grid_resolution
            x_max, y_max = x_min + grid_resolution, y_min + grid_resolution

            # Crop the image to the bounding box
            cell_content = image[y_min:y_max, x_min:x_max]

            # Check if the cell is an obstacle
            if is_black_cell(cell_content):
                map[x][y] = 0
                obstacles.append(Point(x,y))
            else:
                map[x][y] = 1

            # Store the cell content in the grid
            internal_map[x][y] = change_cell(cell_content)

            # Add the cell in the column
            if x==0:
                new_image[y] = internal_map[x][y]
            else:
                new_image[y] = np.hstack((new_image[y], internal_map[x][y]))
        
        # Add the column in the image
        if y==0:
            final_image = new_image[y]

        if y!=0:
            final_image = np.vstack((final_image,new_image[y]))

    # Create the map object
    map = Map([Point(0,0), Point(width,0), Point(width, height), Point(0,height)], obstacles)
    return final_image, map

The map is created when the following line is called in the main (a default grid resolution is provided in case of non detection of the Aruco Marker):

In [ ]:
builtmap, grid_res = apply_grid_to_camera(DEFAULT_GRID_RES)

<div style="text-align: center;">
    <img src="image\captured_frame.png" alt="Setup Map" title="Map" style="width:400px;"/>
</div>
<br>

<div style="text-align:center;">
     captured_frame.png
</div>

#### ArUco detection<a class="anchor" id="arucodetection"></a>

The Aruco markers are detected using the Aruco marker detection functionality provided by the OpenCV library. The cv2.aruco.getPredefinedDictionary function is used to obtain a predefined Aruco marker dictionary, and cv2.aruco.DetectorParameters is employed to configure the marker detection parameters. The cv2.aruco.detectMarkers function is then applied to find and extract Aruco markers' corners and IDs in the camera frame.


Ultimately, we opted for this solution because ArUco enables reliable vision detection, even in challenging lighting situations.

In [ ]:
def set_aruco():
    # Set the dictionary of aruco markers to 4x4_100
    dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_100)
    # Set the parameters for the detector
    parameters =  cv2.aruco.DetectorParameters()
    # Create the detector
    detector = cv2.aruco.ArucoDetector(dictionary, parameters)
    return detector

def get_arucos(frame):
    detector = set_aruco()
    markerCorners, markerIds, rejectedCandidates = detector.detectMarkers(frame)
    frame, arucos = get_info_arucos(markerCorners, markerIds, frame)
    return frame, arucos

#### Thymio detection<a class="anchor" id="thymiodetection"></a>

The ArUco marker 95 is placed on the Thymio. *(What a nice bib, Flash McQueen has the same !)*

The ArUco identification gives us relevant information such as the center coordinates and orientation of the Thymio. By using the ArUco identification we can get information such as the the coordinates of the corners and label them as follow : BottomLeft, BottomRight, TopLeft, and TopRight. This allows us to get the position and orientation of the robot :
- For the center coordinates, it is calculated using the coordinates of 2 of the 4 corners : $(Pos_{x}, Pos_{y})  = (\frac{\text{TopLeft}_x + \text{BottomRight}_x}{2}, \frac{\text{TopLeft}_y + \text{BottomRight}_y}{2})$


- For the angle we use the coordinates of one of the sides of the ArUco : $\alpha = arctan(\frac{TopLeft_{y}-BottomLeft_{y}}{TopLeft_{x}-BottomLeft_{x}})$

In the case the Thymio is not detected (hidden camera for example), it will return a (0,0) position. So the kalman function can continue working without knowing the current localization of the Thymio.

The choice to return this position is due that our reference ArUcO is based at this localisation. So when we are doing the obstacle map, it will be considered such as being an obstacle. The Thymio thus cannot be at (0,0), this position is always considered "Not detected". This allows the kalman filter to keep working without sensing the current localization of the Thymio.

In [ ]:
def show_robot(frame, arucos, grid_resolution):
    real_center = robot_center_is(arucos)
    robot_pos = center_in_grid(arucos, 95, grid_resolution)
    angle = get_angle_of_robot(arucos)
    frame = draw_arrow(frame, arucos, angle)

    return frame, arucos, robot_pos, angle

#### Goal detection <a class="anchor" id="goaldetection"></a>

The goal detection process relies on another ArUco marker (ID n°99) specifically placed to represent the goal location. Similar to Thymio detection, the ArUco marker for the goal is identified in the captured frame. The center coordinates of this marker are extracted to determine the goal's position in the grid.

The variable "last_known_goal_pos" is updated only when the measure goal_position is diffenrent of (0,0). We consider that the goal will not change its position if the camera is hidden. Therefore we are working with the "last_known_goal_pos" variable in the next steps. This is specially useful when the thymio is near to the desired position and starts covering parts of the ArUco goal.

In [15]:
#goal_pos = center_in_grid(arucos, 99, grid_res)

def center_in_grid(arucos, id, grid_resolution):
    if len(arucos) !=0:
        for i in range(len(arucos)):
            if arucos[i][2] == id:
                x = int(arucos[i][0]/grid_resolution)
                y = int(arucos[i][1]/grid_resolution)
                return (x, y)
    return (0, 0)

The variable "last_known_goal_pos" is updated only when the measure goal_position is diffenrent of (0,0). We consider that the goal will not change of position if the camera is hidden. Therefore we are working with the "last_known_goal_pos" variable in the next steps.

#### Obstacles detection <a class="anchor" id="obstaclesdetection"></a>

As explained above, the obstacle detection is based on the assumption that non-white cells in the environment are representing obstacles. 

#### Camera Thread <a class="anchor" id="camerathread"></a>

In the camera thread function *run_camera*, the position of the Thymio and the Goal are only updated when they are none zeros.

Both cannot be located in the (0,0) position in our map, because it is an obstacle. Therefore we know that if we get (0,0) as a position, it means we cannot detect it with the camera.

In [14]:
def run_camera(mes_pos : Robot, mes_goal: Point, camera : Camera, grid_res=DEFAULT_GRID_RES):
    '''
    Function that updates the global Mes_Robot variable with camera data every 0.1 seconds on average

    '''
    cap = cv2.VideoCapture(0)

    ret, frame = cap.read()
    camera.update(ret)
    
    width, height = 707, 10007
    # width, height = set_param_frame(arucos)

    last_known_goal_pos = (0, 0)
    robot_pos = (0, 0)
    while True:
        if cap.isOpened() == False:
            print("Error : video stream closed")
        else:
            frame = cap.read()[1]

            frame, arucos = get_arucos(frame)

            frame, arucos, robot_pos, angle = show_robot(frame, arucos, grid_res)
            goal_pos = center_in_grid(arucos, 99, grid_res)

            if robot_pos != (0,0) :
                robot_pos = Point(robot_pos[0], robot_pos[1])
                mes_pos.update(Robot(robot_pos, angle, True))
            else:
                mes_pos.update(Robot(mes_pos.position, mes_pos.direction, False))
                    
            if goal_pos != (0, 0):
                mes_goal.update(Point(goal_pos[0], goal_pos[1]))
            draw_goal(frame, arucos, grid_res)

            # frame = projected_image(frame, arucos, width, height)
            cv2.imshow("Video Stream", frame)

            # print(f'Robot position: {robot_pos} and angle: {angle}')
            # print(f'Goal position: {goal_pos}')
            
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

        time.sleep(0.095)

### Projection Image <a class="anchor" id="projected-image"></a>

To have a better and more correct view of the map and detect better the positions of the Thymio and the goal, we wanted to extract these informations directly from the projected image.


To do this, first we needed to detect the corners of the map by using 4 ArUcos with different Ids, and get set each of the corners as the center coordinates of these ArUcos. Then we would use cv2.getPerspectiveTransform to get the matric of the transformation applied to the image and then use the cv2.warpPerspective method to have the projection image.
<br><br>
<div style="display: flex; justify-content: center; align-items: center;">
    <div style="text-align: center;">
        <img src=image/projected_image.png height=400px alt="Image 1" >
        <p>Projection Image</p>
    </div>
    <div style="text-align: center;">
        <img src=image/non_projected_image.png height=400px alt="Image 2" >
        <p>Original Image</p>
    </div>
</div>
<br><br>
As you can see above, we can access infromation about the Thymio ArUco in the original image, but not in the projection image. We are not sure why exactly it is not possible to detect the Thymio in the first image, but we suspect it is due to distortion however slightly it might seem. We tried several techniques to resolve this issue, like making sure the resulted image having the same aspect ratio as the paper we use in the origial image, but it still wouldn't work properly. Another idea was to collect the data of the thymio from the orginal image and apply the same matrix of transformation to get the coordinates in the projection image, but this was not very stable to be used in our model.

In the end, we opted for a more traditional way of collecting information from the camera, but still assuring a correct way of doing so by positioning the camera well centered above the map to have the least errors in collecting the data needed.
<br>

<br>
<div style="text-align: center;">
    <img src="image/setup_map.jpg" alt="Setup Map" title="Map" style="width:400px;"/>
    <p>Well adjusted camera view</p>
</div>



### Results <a class="anchor" id="results"></a>

The results are in the majority of the time good. Our biggest issue is actually the time it takes to the camera to connect to the computer. Therefore we put waiting nodes, to let everything get in place before launching the "race". 

When the calibrating ArUco is not detected (too much noise or bad light conditions), we can observe that obstacles are not perfectly aligned on the grid (with is due to the grid resolution by default). 

To test the vision, you can fetch and run the file **test_thread_cam.py" in Codes. It will show the obstacle map, the localization of the Thymio and the goal.

Press on 'p' to stop the videos, and kill the terminal.

-------

## Path planning<a class="anchor" id="pathplanning"></a>

The goal of the path planning is to find the fastest, so the shortest, path for Thymio to reach the final line. It takes as input the visibility graph of the map, and returns the optimal path, which is a list of coordinates for Santo to follow. 

Where do we get our informations ?

Thanks to the previous steps, we get a global map composed of white and black cells ("capture_frame.jpg") that enables us to know where the thymio can go through.

The start and end positions are defined by the initial position of the Thymio (aruco 95) and the goal (aruco 99) last detected position "last_known_goal_pos".

The Path Planning algorithm used was inspired by the A* algorithm seen in the course excercise session $^{[1]}$.

Path Following Module: the estimation of the robot's position is given by the kalman_filter function.

### Global navigation <a class="anchor" id="globalnavigation"></a>

We decided to use the A* algorithm for the path planning. The goal is to find the optimal path from a starting point to a goal point, considering obstacles in the environment.

Prior to computing the optimal path, we add half of thymio's length to all obstacles, so that the Thymio, considered as a point, won't run into it. Now we can calculate the path using the A* algorithm.

#### A* algorithm <a class="anchor" id="Aalgorithm"></a>


We chose to use the A* algorithm because it optimizes the search of the optimal path by expanding the most promising node chosen according some cost and heuristic functions.

<div style="text-align: center;">
    <img src="image\A-algorithm-explanation.png" alt="A algo slide" title="A * algorithm" style="width:400px;"/>
</div>


<div style="text-align:center;">
    "A* algorithm, extending Dijkstra" [1]
</div>

Functions:

$f(n)=g(n)+h(n)$

$f(n)$: The f-score of node nn

$g(n)$: The cost of the cheapest path from the start node to node nn

$h(n)$: The heuristic estimate of the cost from node nn to the goal


The calculate_path function orchestrates the entire path planning process presented below:

In [2]:
def calculate_path(env, finalpath, extended_obs, visitedNodes, size_thym, PLOT=False):
    """calls all functions to calculate path
       out: path given in a list of Points 
    """

    grid, start, goal = create_grid(env)                 #Converts useful infos from environment
    occupancy_grid = map_without_collision(grid, extended_obs, size_thym)     #add half the size of Thymio
    max_x, max_y = grid.shape[0], grid.shape[1]          # Size of the map
    # List of all coordinates in the grid
    x,y = np.mgrid[0:max_x:1, 0:max_y:1]
    pos = np.empty(x.shape + (2,))
    pos[:, :, 0] = x; pos[:, :, 1] = y
    pos = np.reshape(pos, (x.shape[0]*x.shape[1], 2))
    coords = list([(int(x[0]), int(x[1])) for x in pos])

    # Define the heuristic, here = euclidian distance to goal ignoring obstacles
    h = np.linalg.norm(pos - goal, axis=-1)
    h = dict(zip(coords, h))

    # Run the A* algorithm
    path, closedSet = A_Star(start, goal, h, coords, occupancy_grid, visitedNodes)
    path = np.array(path).reshape(-1, 2).transpose()

    closedSet = np.array(closedSet).reshape(-1, 2).transpose()

    convert_path(path, finalpath)                  #convert to a list of Points

* **Initialization:**
    The algorithm starts by checking if the start and goal points are within the boundaries of the map and whether they are in a traversable space. If not, an exception is raised.
    The possible movements are defined based on 8-connectivity, meaning the algorithm considers movements in eight possible directions (horizontal, vertical, and diagonal).
    The A* algorithm uses sets (openSet and closedSet) and dictionaries (cameFrom, gScore, and fScore) to keep track of nodes and their associated scores.
    
    
    For node n, cameFrom[n] is the node immediately preceding it on the cheapest path from start to n currently known.

    For node n, gScore[n] is the cost of the cheapest path from start to n currently known.
    
    For node n, fScore[n] = gScore[n] + h(n). map with default value of Infinity

#### Implementation <a class="anchor" id="Aalgorithm"></a>


* **A-Algorithm Implementation:**
    The A* algorithm is implemented using a while loop that continues until there are no more nodes to explore (openSet is empty) or the goal is reached.
    In each iteration, the algorithm selects the node with the lowest fScore from the openSet. This node is set as the current node.
    The neighbors of the current node are explored, and for each neighbor, the algorithm calculates tentative scores (gScore and fScore).
    If a neighbor is not in the openSet, it is added. If the tentative gScore for a neighbor is lower than its current gScore, the algorithm updates the scores and backtracks the optimal path.
    The algorithm continues to explore nodes until the goal is reached or all reachable nodes have been explored.

* **Reconstruction of Path:**
    If the goal is reached, the *reconstruct_path* function is called to reconstruct the optimal path from the start to the goal using the information stored in the cameFrom dictionary.
    The reconstructed path is returned along with the set of nodes that were visited during the algorithm (closedSet).

* **Visualization (Optional):**
    The code includes optional visualization elements, such as displaying the visited nodes with a time delay 0f 0.05 seconds, to visualize the algorithm's progress.

* **Error Handling:**
    If the open set becomes empty and the goal has not been reached, a message is printed indicating that no path was found to the goal.


In [23]:
def calculate_path(env, finalpath, extended_obs, visitedNodes, size_thym, PLOT=False):
    """calls all functions to calculate path
       out: path given in a list of Points 
    """

    grid, start, goal = create_grid(env)
    occupancy_grid = map_without_collision(grid, extended_obs, size_thym)
    max_x, max_y = grid.shape[0], grid.shape[1] # Size of the map
    # List of all coordinates in the grid
    x,y = np.mgrid[0:max_x:1, 0:max_y:1]
    pos = np.empty(x.shape + (2,))
    pos[:, :, 0] = x; pos[:, :, 1] = y
    pos = np.reshape(pos, (x.shape[0]*x.shape[1], 2))
    coords = list([(int(x[0]), int(x[1])) for x in pos])

    # Define the heuristic, here = distance to goal ignoring obstacles
    h = np.linalg.norm(pos - goal, axis=-1)
    h = dict(zip(coords, h))

    # Run the A* algorithm
    path, closedSet = A_Star(start, goal, h, coords, occupancy_grid, visitedNodes)
    path = np.array(path).reshape(-1, 2).transpose()

    closedSet = np.array(closedSet).reshape(-1, 2).transpose()

    convert_path(path, finalpath)

#### Results <a class="anchor" id="A*CCL*"></a>

We are happy with the results of our algorithm. It can very well compute the optimal path through the map an raise alerts when either the goal or the thymio are in obstacles. 
The main limit is the time it takes to compute the path especially when the goal is far away. This limit also constrains the grid resolution we can pick for time to remain acceptable.

You can see the algorithm process in the display windows when the Thymio is planning it's path.

<div style="text-align: center;">
    <img src="image\map_path_compute.jpg" alt="pathcomputation" title="Results path computation" style="width:750px;"/> 
</div>


<div style="text-align:center;">
     "Results of the optimal path computation"
</div>

 
The goal is labeled in red, and our thymio has a green arrow on it.


-------


# Kalman Filtering <a class="anchor" id="filtering"></a>

We decided to work with a Kalman filter because it is optimal for the robot localization due to its ability to handle noise and uncertainties effectively. Indeed, Kalman filters are used to estimate the state of a system based on a series of noisy and/or incomplete measurements.

Therefore, the Kalman filter proves to be very useful when the signal between the Thymio and the camera is lost. For instance when the camera is hidden, or because it does only detect the ArUcO periodically due to noisy images.

Furthermore, Kalman filters use a recursive estimation process, meaning that they continually update the state estimate as new measurements become available. This allows the filter to provide real-time position estimates, even if the camera is hidden.

We need to gather informations about the thymio's detected state:
- overhead camera localization
- odometry and thymio's target speed

### Mathematical model <a class="anchor" id="mathmodel"></a>


The Kalman filtering is composed of 3 main parts : 

1. *Prediction through motion model:*
    * A priori state estimate 
    * A priori covariance estimate    
2. *Measurements through measurement model:*
    * Check for localization measurement
    * Update sensed motors speeds
    * Compute Measurement matrix C
3. *State Update:*
    * Innovation or measurement residual
    * Kalman gain computation
    * A posteriori state estimate
    * A posteriori covariance estimate 

The Thymio's movement is describe by its translation on the x and y axes and its rotation $\theta$. 
 

$$X = 
\begin{bmatrix}
v\\
\omega\\
p_x\\
p_y\\
\theta
\end{bmatrix}
$$
Where:
- $v$ is the magnitude speed in $[cases/s]$.
- $\omega$ is the angular speed $[rad/s]$.
- $p_x$, $p_y$ is the cartesian position in $[cases]$.
- $\theta$ is the orientation in $[rad]$.

*Note : one $case$ is defined by grid resolution pixels* 

Our input is $U = [dmot_l, dmot_r]$, the delta between the new and the previous motors speed targets.
Our measurement can either be $[Smot_l, Smot_r]$ the sensed motors speed if the camera don't detect the Thymio, or $[Smot_l, Smot_r, x, y, \theta]$ with the measured robot localisation otherwise

### Prediction step <a class="anchor" id="prediction"></a>

Our Motion model is ruled by the following laws :

1. Forward speed :    $v_{+} = v_{previous} + K*\frac{dmot_R + dmot_L}{2}$
2. Rotational speed : $w_{+} = w_{previous} + K*\frac{dmot_R - dmot_L}{width}$
3. Position :         $x_{+} = x_{previous} + v \cdot \cos(\theta) \cdot dt$
4. Position :         $y_{+} = y_{previous} + v \cdot \sin(\theta) \cdot dt$
5. Direction :        $\theta_{+} = \theta _{previous} + w \cdot dt$ 

$\theta$ is kept in range of $[-\pi,\pi]$ with a modulo function

K being a conversion factor such that 1 thymio's speed equals K x 1 case per second

Our model is thus non linear due to theta and can't work with the classical A & B matrices : 
$x_{+} = Ax + Bu + w$ 

We thus have to compute the Jacobian of our model to update our covariance matrix. 

This gives :
$J = \begin{bmatrix}
1.0 & 0 & 0 & 0 & 0 \\
0 & 1.0 & 0 & 0 & 0 \\
\cos(\theta) \cdot dt & 0 & 1.0 & 0 & -v \cdot \sin(\theta) \cdot dt \\
\sin(\theta) \cdot dt & 0 & 0 & 1.0 & v \cdot \cos(\theta) \cdot dt \\
0 & dt & 0 & 0 & 1.0
\end{bmatrix}$

and then our a priori covariance : $P_{\text{prior}} = J P_{\text{post}} J^T + Q$   
with Q being the covariance matrix of the state stochatic noise 


### Measurement step <a class="anchor" id="prediction"></a>

Our Measurement Model depends whether we have a camera measure of the robot localization or not. 

The measurement model is defined by the matrix $C$. If the camera measurement is not found, $C$ is a $2 \times 5$ matrix (with K defined as previous):

$
C = \begin{bmatrix}
\frac{1}{K} & -\frac{1}{K} & 0 & 0 & 0 \\
\frac{1}{K} & \frac{1}{K} & 0 & 0 & 0
\end{bmatrix}
$

In this case, the measured state $y$ is a $2 \times 1$ vector containing the left and right motor speeds:

$
Y = \begin{bmatrix}
\text{Smot}_{\text{left}} \\
\text{Smot}_{\text{right}}
\end{bmatrix}
$

If the camera measurement is found, $C$ is a $5 \times 5$ identity matrix, and the measured state $y$ is a $5 \times 1$ vector containing the left and right motor speeds, the x and y positions, and the direction:

$
C = \begin{bmatrix}
\frac{1}{K} & -\frac{1}{K} & 0 & 0 & 0 \\
\frac{1}{K} & \frac{1}{K} & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1
\end{bmatrix}
$

$
Y = \begin{bmatrix}
\text{Smot}_{\text{left}} \\
\text{Smot}_{\text{right}} \\
\text{x}_{\text{mes}} \\
\text{y}_{\text{mes}} \\
\text{angle}_{\text{mes}}
\end{bmatrix}
$

In both cases, the measurement model is defined such that $y = Cx$, where $x$ is the true state of the system.

### Update step <a class="anchor" id="prediction"></a>

First, the Kalman gain $K$ and the innovation are computed:

$
K = P_{\text{prior}} C^T (C P_{\text{prior}} C^T + R)^{-1}
$

$
\text{innovation} = y - C x_{\text{prior}}
$

where $P_{\text{prior}}$ is the a priori state covariance, $C$ is the measurement model, $R$ is the measurement noise covariance, $y$ is the measured state, and $x_{\text{prior}}$ is the a priori state.


Next, the a posteriori state and covariance are estimated:

$
x_{\text{post}} = x_{\text{prior}} + K \cdot \text{innovation}
$

$
P_{\text{post}} = (I - K C) P_{\text{prior}}
$

where $I$ is the identity matrix.

The estimated speed, position, and covariance are then stored for use in the next iteration of the filter.

### Testing and simulation <a class="anchor" id="filterconcl"></a>

This Kalman filter has been partly tested and tuned through a simulation loop that simulates noisy inputs and measurements such as this one:

In [ ]:
inputspeed=np.array([50,50])
for i in range(steps):
    input = Motors(inputspeed[0], inputspeed[1])
    mot_mes = Motors(np.random.normal(inputspeed[0],speedvar), np.random.normal(inputspeed[1],speedvar))

    pos_next, speed = kalman.kalman_filter(input, mot_mes, pos)

    xnext = np.random.normal(pos_next.position.x + speed[0]*np.cos(pos_next.direction)*dt, posvar)
    ynext = np.random.normal(pos_next.position.y + speed[0]*np.sin(pos_next.direction)*dt, posvar)
    thetanext = np.random.normal(pos_next.direction + speed[1]*dt, thetavar)
    pos = Robot(position=Point(x=xnext, y=ynext), direction=thetanext, found = Cam)
    
    memory.append([pos_next.position.x, pos_next.position.y, pos_next.direction])

We defined the variances of our noise by measuring the actual ones on the thymio and camera giving us the following measurement noise matrix $R : 

$
R = \begin{bmatrix}
\text{MOT\_VAR} & 0 & 0 & 0 & 0 \\
0 & \text{MOT\_VAR} & 0 & 0 & 0 \\
0 & 0 & \text{POS\_VAR} & 0 & 0 \\
0 & 0 & 0 & \text{POS\_VAR} & 0 \\
0 & 0 & 0 & 0 & \text{DIR\_VAR}
\end{bmatrix}
$

The process noise covariance matrix $Q$ has been set after various tries to :

$
Q = \begin{bmatrix}
1 & 0 & \text{dt} & \text{dt} & 0 \\
0 & \frac{2}{\text{WIDTH}} & 0 & 0 & \text{dt} \\
\text{dt} & 0 & 1 & 0 & 0 \\
\text{dt} & 0 & 0 & 1 & 0 \\
0 & \text{dt} & 0 & 0 & 1
\end{bmatrix}
$

This gives us satisfaction in the simulation even with very noisy inputs & measurements. This example is a result of a run of the Test_Kalman.py file : 

<div style="text-align:center;">
        <img src="image/noisy direction.png" alt="Dir Plot"  style="width:400px;"/>
        <img src="image/noisy_position.png" alt="Pos Plot"  style="width:400px;"/>
</div>



### Results <a class="anchor" id="filterconcl"></a>

The extended Kalman filter satifies our needs for this project. It allows to merge the camera measure and to situate our robot even without camera.
It could still uses some tuning as he adds a little offset to the real position that we can see in the video provided (Kalman estimated position on the display and real position on the real time feed) 



-----

## Motion Control <a class="anchor" id="motioncontrol"></a>

To compute the speed of the Thymio, we decided to decompose the computation in three parts: orientation, position and obstacle detection.
The final velocity will be composed of the three different speeds: $\vec{v}_{total}=\vec{v}_{orientation}+\vec{v}_{forward}+\vec{v}_{obstacle}= \begin{bmatrix} v_L \\ v_R \end{bmatrix}$. We got inspired by this [article](https://cooprobo.readthedocs.io/en/latest/mobile/pioneer/model/kinematics.html).


<div style="text-align:center;">
    <img src="image/graph_motion.jpg" alt="Motion Control" title="A robot is moving around with the proposed motion framework" style="width:300px;"/>
</div>

<div style="text-align: center;">
   A robot is moving around with the proposed motion framework 
</div> 

### Orientation & angle error <a class="anchor" id="velorientation"> </a>

We define orientation changes as an angle error defined by: $\alpha _E= \alpha _C - \alpha _T$, where $\alpha _T$ is the current angle of the thymio and $\alpha _C$ is the desired angle change.

With classic trigonometry formel we get that: $\alpha _C = arctan(\frac{P_{y+1}-P_{yT}}{P_{x+1}-P_{xT}})$, where $P_T$ the position of the Thymio and $P_{+1}$ the desired position of the Thymio.


One of the big limitations found is because of the use of the angles functions. Indeed, it gives values between $[0; 2\pi]$. So if in some cases we where to near of 0 or $2\pi$, the robot just turns around infinitly. 
Therefore we decided to normalize all angles between $[-\pi; \pi]$, to prevent discontinuities and unexpected behavior.


In [11]:
def compute_angle_error(position, checkpoint, thymio_direction):
  """
  Computes the angle error between the robot and the goal.

  :param position: Current position of the robot.
  :param checkpoint: position to reach (next point in the global path).
  :param thymio_angle: Current orientation of the robot.
  :return: Angle error between the robot and the goal.
  """
  angle = np.arctan2( (checkpoint.y - position.y), (checkpoint.x - position.x))
  # angle = (angle + np.pi) % (2 * np.pi) - np.pi

  thymio_direction = (-thymio_direction + np.pi) % (2 * np.pi) - np.pi    ## + thymio_direction because it's defined in the negative sense of trigo

  angle_error = angle - thymio_direction         
  angle_error = (angle_error + np.pi) % (2 * np.pi) - np.pi

  print("angle: ", angle, "thymio_direction: ", thymio_direction, "angle_error: ", angle_error)

  return angle_error



def compute_derived_angle_error(angle_error, prev_angle_error, dt):
  """
  Computes the derived angle error between the robot and the goal.

  :param angle_error: Angle error between the robot and the goal.
  :param prev_angle_error: Previous angle error between the robot and the goal.
  :param dt: Time step.
  :return: Derived angle error between the robot and the goal.
  """
  return (angle_error - prev_angle_error) / dt

### PD Controler<a class="anchor" id="controler"> </a>

We decided to use a PD controler because of it is simple to implement and it is suitable for fast, real-time feedback.

First, we only had a $K_p$ controler, we could see that the Thymio made some oscillations in its motion, therefore we added the $K_d$ to counteract those undesired movement and so get a more stable response. The derivative helps to reduce the sensitivity of the noise.

$K_{p}$ and $K_d$ are constant.

### Compute velocity <a class="anchor" id="veldirection"> </a>


* *'base_speed'* is the forward velocity that you want the robot to maintain.

$v_{forward}=const.$

The other calculated speeds are going to act as a penalty or gain on top of the *'base_speed'* for each wheel. 

* *'rot_speed'* is the rotational speed calculated using the PD controler based on the angle error (*'angle_error'*) and its derivative (*'derived_angle_error'*).

$\vec{v}_{orientation}=(K_{p} * \alpha _E + K_{d} * \frac{d \alpha _E}{dt} )\begin{bmatrix} 1 \\ -1 \end{bmatrix}$.

The [1,-1] matrix is because to turn, the two wheels need to have opposite speeds.


$\vec{v}_{wheel}= \vec{v}_{forward} + \vec{v}_{orientation}$


The rotational speed is clipped to avoid saturation (>500). If the absolute value of rot_speed exceeds the limit 200 - base_speed, it is adjusted to this limit to prevent oversaturation and the program to crash.



In [10]:
def controller(robot, checkpoint, base_speed, prev_angle_error, dt, final_goal_reached=False):
  '''
  This function computes the motor velocities based on the position, goal, and control parameters.
  param robot : Current position of the robot
  param checkpoint : Goal position to reach (next point in the global path)
  param base_speed : Base speed of the robot
  param final_goal_reached : Flag indicating whether the final goal is reached, not motion is needed.

  return : Input as a tuple (MotL, MotR) 
  return : angle_error
  '''
  if final_goal_reached:
      return 0, 0, 0
  
  angle_error = compute_angle_error(robot.position, checkpoint, robot.direction)
  derived_angle_error = compute_derived_angle_error(angle_error, prev_angle_error, dt)

  rot_speed = Kp * angle_error + Kd * derived_angle_error

  # Clip the rotation speed to avoid saturation
  rot_speed = rot_speed if abs(rot_speed) < 200 - base_speed else 200 - base_speed

  MotL = base_speed + rot_speed
  MotR = base_speed - rot_speed

  return MotL, MotR, angle_error

### Local navigation <a class="anchor" id="localnavigation"> </a>

First we wanted to use the potential field Navigation system and totaly ignoring the other velocities when an obstacle is encountered. It came quickly out that the thymio easily gets out of the desired path. Therefore, we tried to do something similar, but the $\vec{v}_{obstacle}$ will act as a gain on the total computed speed $\vec{v}_{wheel}$. We used the same gains for the proximity sensors as in the exercise session 4 $^{[1]}$.

The external sensors have a stronger weight than the internal ones. The Thymio will be 'repulsed' more quickly when it detects an obstacle, so it will no longer be in line with it.

The code below is similar as the one used in the merge version of the code, to make it work independently.

In [16]:
#if needed decomment:

import tdmclient.notebook
await tdmclient.notebook.start()

In [20]:
%%run_python
timer_period[0] = 10  # 10ms sampling time

@onevent 
def timer0():
    global prox_horizontal, motor_left_target, motor_right_target
    speed0 = 75     # nominal speed

    # gains used with front proximity sensors [0,5]
    weights = [6, 4, -3, -6, -8]

    addLeft=0
    addRight=0
    diffDelta=0
    # adjustment for obstacles, "gradient" due to obstacles
    for i in range(5):
        addLeft += prox_horizontal[i] * weights[i]//200
        addRight += prox_horizontal[i] * weights[4 - i]//200
    
    motor_left_target = speed0 + addLeft
    motor_right_target = speed0 + addRight

<div style="text-align:center;">
<video width="640" height="360" controls autoplay>
  <source src="image/local_avoidance.mp4" type="video/mp4">
</video>
</div>

### Results <a class="anchor" id="MCresults"> </a>

The motion control works well when the parameters are tuned, the speed could be improved but this implementation assures the inputs given for the motor never reach the threshold. 

-----

## Conclusion <a class="anchor" id="conclusion">

To conclude, we have achieved all the given tasks and are satisfied with our results. We tried differents approaches to solve each part which took some time to reach concluant results, especially for the vision part before using Arucos as the light conditions influences the efficiency. We saw some limits of our code: we implemented a projection function at the begining to avoid distortions on the map but it brought some problems in the detection of Aruco afterward, we have therefore done without as it doesn't bring much error. The choice of a grid for the mapping also brought some computational problem for bigger maps. Using the coordinate system of the image was a challenge as it is non conventional for motion and needed conversions.

### Sources :


[1] MICRO-452 "Basics of mobile robotics" course of EPFL from Professor Francesco Mondada

[2] « Kinematics — CoopRobo 1.0.0 documentation » https://cooprobo.readthedocs.io/en/latest/mobile/pioneer/model/kinematics.html

[3] « Extended Kalman Filter » Wikipedia https://en.wikipedia.org/w/index.php?title=Extended_Kalman_filter&oldid=1185257110.
